In [597]:
import pandas as pd
import numpy as np

## Simulate Games to calculate xP

Basic strategy: 

- simulate each shot N times
- sum each of those shots along the columns by event.id, by team
- compare sum by team by simulation
- calculate the probability the team wins the event by comparing % of matches wrt win, lose, draw

In [598]:
# import the shots dataset
shots = pd.read_csv("../datasets/23_24_shotmaps_augmented.csv", index_col='Unnamed: 0')

In [599]:
# pull P(Goal) and match info
sim_df = shots[['xG', 'team', 'opponent', 'event.id']]

In [600]:
# find the number of sample, and the associated weights
N_samples = 10_000
shot_sim_weights = sim_df[['xG']].values

In [601]:
# numeric_columns = [f'Simulation_{i + 1}' for i in range(N_samples)]

In [602]:
# simulate each of the shots
shot_sim_matrix = np.random.binomial(1, shot_sim_weights.squeeze(), size=(N_samples, shot_sim_weights.shape[0]))
shot_sim_matrix.shape

(10000, 1121)

In [603]:
# create column names for the simulated df and concat with match info
simulated_shots_df = pd.DataFrame(shot_sim_matrix.T, columns=[f'Simulation_{i+1}' for i in range(shot_sim_matrix.shape[0])])

simulated_shots_df = pd.concat([simulated_shots_df, shots[['event.id', 'team', 'opponent']]], axis=1)
simulated_shots_df.head()

,Simulation_1,Simulation_2,Simulation_3,Simulation_4,Simulation_5,Simulation_6,Simulation_7,Simulation_8,Simulation_9,Simulation_10,...,Simulation_9994,Simulation_9995,Simulation_9996,Simulation_9997,Simulation_9998,Simulation_9999,Simulation_10000,event.id,team,opponent
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,11352251,Arsenal,Nottingham Forest
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,11352251,Nottingham Forest,Arsenal
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,11352251,Arsenal,Nottingham Forest
3,1,0,0,0,1,1,0,0,1,1,...,0,0,0,0,1,0,1,11352251,Arsenal,Nottingham Forest
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,11352251,Arsenal,Nottingham Forest


In [604]:
# sum each column by event, opponent and team; take the transpose
simulated_events_df = simulated_shots_df.groupby(['event.id', 'team', 'opponent']).sum()
simulated_events_df_t = simulated_events_df.T
simulated_events_df_t.head()

event.id            11352250                          11352251  \
team             Bournemouth West Ham United           Arsenal   
opponent     West Ham United     Bournemouth Nottingham Forest   
Simulation_1               1               2                 4   
Simulation_2               1               3                 0   
Simulation_3               1               3                 4   
Simulation_4               1               1                 2   
Simulation_5               3               1                 4   

event.id                                     11352252                         \
team         Nottingham Forest Brighton & Hove Albion             Luton Town   
opponent               Arsenal             Luton Town Brighton & Hove Albion   
Simulation_1                 0                      4                      1   
Simulation_2                 1                      4                      1   
Simulation_3                 2                      2                      1   
Simulation_4                 1                      5                      1   
Simulation_5                 1                      3                      0   

event.id     11352253                 11352254                   ...  \
team          Everton  Fulham   Crystal Palace Sheffield United  ...   
opponent       Fulham Everton Sheffield United   Crystal Palace  ...   
Simulation_1        1       2                1                0  ...   
Simulation_2        3       2                3                0  ...   
Simulation_3        3       2                2                0  ...   
Simulation_4        4       1                3                0  ...   
Simulation_5        2       1                2                0  ...   

event.id           11352578                   11352584              \
team         Crystal Palace  Wolverhampton Aston Villa   Liverpool   
opponent      Wolverhampton Crystal Palace   Liverpool Aston Villa   
Simulation_1              4              0           0           3   
Simulation_2              2              0           0           5   
Simulation_3              6              0           0           4   
Simulation_4              3              0           0           3   
Simulation_5              2              2           0           4   

event.id            11352590                        11352594                  \
team              Luton Town West Ham United          Fulham Manchester City   
opponent     West Ham United      Luton Town Manchester City          Fulham   
Simulation_1               2               2               0               4   
Simulation_2               0               0               1               3   
Simulation_3               1               1               1               3   
Simulation_4               0               1               1               4   
Simulation_5               2               2               1               3   

event.id             11352597                   
team                  Everton Sheffield United  
opponent     Sheffield United          Everton  
Simulation_1                2                3  
Simulation_2                1                1  
Simulation_3                4                2  
Simulation_4                2                1  
Simulation_5                3                1  

[5 rows x 78 columns]

In [605]:
# calculate possible outcomes for each event
simulated_results = []
for event_id, team, opponent in simulated_events_df_t.columns:
    simulated_results_dict = {'event.id': event_id, 
    # 'slug': f'{team}-{opponent}',
    'team': team,
    'opponent': opponent,
    'results': np.array(simulated_events_df_t[event_id][team][opponent] - simulated_events_df_t[event_id][opponent][team])}

    simulated_results_dict['outcome_code'] = np.where(simulated_results_dict['results'] > 0, 1, -1)
    simulated_results_dict['outcome_code'] = np.where(simulated_results_dict['results'] == 0, 0, simulated_results_dict['outcome_code'])

    simulated_results_dict['win.pct'] = np.average(np.where(simulated_results_dict['outcome_code'] == 1, 1, 0))
    simulated_results_dict['draw.pct'] = np.average(np.where(simulated_results_dict['outcome_code'] == 0, 1, 0))

    simulated_results_dict['xP'] = 3 * simulated_results_dict['win.pct'] + 1 * simulated_results_dict['draw.pct']

    simulated_results.append(simulated_results_dict)

In [606]:
simulated_results_df = pd.DataFrame(simulated_results)
simulated_results_df.head()

,event.id,team,opponent,results,outcome_code,win.pct,draw.pct,xP
0,11352250,Bournemouth,West Ham United,"[-1, -2, -2, 0, 2, -2, 0, 2, 2, 2, -1, -1, 0, ...","[-1, -1, -1, 0, 1, -1, 0, 1, 1, 1, -1, -1, 0, ...",0.4030,0.3190,1.5280
1,11352250,West Ham United,Bournemouth,"[1, 2, 2, 0, -2, 2, 0, -2, -2, -2, 1, 1, 0, 1,...","[1, 1, 1, 0, -1, 1, 0, -1, -1, -1, 1, 1, 0, 1,...",0.2780,0.3190,1.1530
2,11352251,Arsenal,Nottingham Forest,"[4, -1, 2, 1, 3, 2, 0, 0, 3, 3, 3, 1, 3, 4, 1,...","[1, -1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,...",0.7602,0.1763,2.4569
3,11352251,Nottingham Forest,Arsenal,"[-4, 1, -2, -1, -3, -2, 0, 0, -3, -3, -3, -1, ...","[-1, 1, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1, ...",0.0635,0.1763,0.3668
4,11352252,Brighton & Hove Albion,Luton Town,"[3, 3, 1, 4, 3, 5, 4, 4, 6, 3, 4, 6, 2, 3, 3, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.9275,0.0560,2.8385


In [607]:
xp_table = pd.DataFrame(simulated_results_df.groupby(by='team')['xP'].sum()).sort_values(by='xP', ascending=False).reset_index()
xp_table.index = range(1, xp_table.shape[0] + 1)
xp_table['xP'] = xp_table['xP'].map('{:,.3f}'.format).astype('float64')
xp_table = xp_table.rename(columns={'team': 'Team'})

In [608]:
xp_table

,Team,xP
1,Manchester City,10.869
2,Arsenal,9.343
3,Tottenham Hotspur,9.248
4,Crystal Palace,7.746
5,Brighton & Hove Albion,7.549
6,West Ham United,7.528
7,Liverpool,7.034
8,Brentford,6.563
9,Chelsea,6.492
10,Aston Villa,5.884


In [609]:
xg_table = pd.read_csv('../assets/xG_table.csv', index_col='Unnamed: 0')

full_table = xg_table.merge(xp_table, how='outer', left_on='Team', right_on='Team')
full_table = full_table.sort_values(by='xP', ascending=False, inplace=False)
full_table.index = range(1, full_table.shape[0] + 1)

In [610]:
full_table.to_csv('../assets/full_table.csv')
full_table

,Team,xG,xGA,xG Differential,xP
1,Manchester City,10.554,2.331,8.223,10.869
2,Arsenal,8.231,3.337,4.893,9.343
3,Tottenham Hotspur,10.405,5.023,5.382,9.248
4,Crystal Palace,6.063,2.627,3.436,7.746
5,Brighton & Hove Albion,9.638,6.411,3.227,7.549
6,West Ham United,8.264,5.394,2.871,7.528
7,Liverpool,8.190,5.419,2.772,7.034
8,Brentford,7.941,5.425,2.517,6.563
9,Chelsea,6.414,4.537,1.877,6.492
10,Aston Villa,7.865,8.311,-0.446,5.884
